In [24]:
import pandas as pd
import ast
import pyarrow
import json
import gzip
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq


In [25]:
games= pd.read_parquet(r'C:\Users\Moises\Desktop\Projecto invidual 1\archivos limpios\games.parquet')
df = pd.read_parquet(r'C:\Users\Moises\Desktop\Projecto invidual 1\archivos limpios\games.parquet')
reviews = pd.read_parquet(r'C:\Users\Moises\Desktop\Projecto invidual 1\archivos limpios\reviews.parquet')
item_user = pd.read_parquet(r'C:\Users\Moises\Desktop\Projecto invidual 1\archivos limpios\items.parquet')
data_final =pd.read_parquet(r'C:\Users\Moises\Desktop\Projecto invidual 1\archivos limpios\data_final.parquet')

In [26]:
df_1=games[['developer','release_date','price','title']]
df_1=df_1.drop_duplicates(subset=['developer','title']).copy()

In [27]:
def developer(developer: str):
    if developer not in list(df['developer']):
        return {'El desarrollador no se encuentra en la base de datos'}

    fechas = df['release_date'].unique()  # --- > Obtiene las fechas únicas de lanzamiento

    anio = {} 
    free = {} 

    for x in fechas:
        filter_condition = (df['release_date'] == x) & (df['developer'] == developer) 
        developer_releases = df[filter_condition]  # --- > Filtra los lanzamientos del desarrollador

        if len(developer_releases) != 0:
            anio[x] = len(developer_releases)  # --- > Almacena la cantidad de lanzamientos del desarrollador
            free[x] = len(developer_releases[developer_releases['price'] == 0.0])  # --- > Almacena la cantidad de lanzamientos gratuitos

    for x, y in free.items():
        free[x] = f"{round((y / anio[x]) * 100, 2)}%"  # --- > Calcula el porcentaje de lanzamientos gratuitos respecto al total por año

    # --- > Convierte las claves del diccionario 'anio' y 'free' a cadenas
    anio = {str(k): v for k, v in anio.items()}
    free = {str(k): v for k, v in free.items()}

    resultado = {'Cantidad de Items': anio, 'Contenido Free': free} 

    return resultado  # --- > Devuelve el resultado

In [28]:
developer('Valve')

{'Cantidad de Items': {'2017': 3,
  '1998': 1,
  '2016': 1,
  '2006': 2,
  '2005': 1,
  '2003': 1,
  '2007': 4,
  '2000': 2,
  '2001': 1,
  '2004': 5,
  '1999': 1,
  '2008': 1,
  '2009': 1,
  '2010': 2,
  '2011': 2,
  '2012': 3,
  '2014': 9},
 'Contenido Free': {'2017': '0.0%',
  '1998': '0.0%',
  '2016': '100.0%',
  '2006': '0.0%',
  '2005': '100.0%',
  '2003': '0.0%',
  '2007': '50.0%',
  '2000': '0.0%',
  '2001': '0.0%',
  '2004': '0.0%',
  '1999': '0.0%',
  '2008': '0.0%',
  '2009': '0.0%',
  '2010': '50.0%',
  '2011': '0.0%',
  '2012': '66.67%',
  '2014': '100.0%'}}

In [29]:
df_2=data_final[['user_id','title','price','items_count','recommend']]
df_2=df_2.drop_duplicates(subset=['user_id','title'], keep='first').copy()

In [30]:
def user_info(user: str):
    try:
        if user not in df_2['user_id'].unique():
            return {f'El usuario {user}, no existe.'}
        else:
            df=df_2[df_2['user_id']==user]
            resultado = {
                'Usuario': user,
                'Dinero gastado': df['price'].sum(),
                'cantidad de items': df['items_count'].head(1).to_list()[0],
                'Porcentaje de recomendaciones': f'{float(round((df['recommend'].sum()/df["items_count"].head(1).to_list()[0])*100,2))}%'
            }

        return resultado
    except:
        return {'El usuario no se encuentra en la base de datos.'}

In [31]:
usuarios_2=df_2['user_id'].unique()
diccionario={}
for i in usuarios_2[:5]:
    diccionario[i]=user_info(i)
diccionario

{'76561197970982479': {'Usuario': '76561197970982479',
  'Dinero gastado': 29.979999999999997,
  'cantidad de items': 277,
  'Porcentaje de recomendaciones': '0.72%'},
 'js41637': {'Usuario': 'js41637',
  'Dinero gastado': 29.979999999999997,
  'cantidad de items': 888,
  'Porcentaje de recomendaciones': '0.23%'},
 'evcentric': {'Usuario': 'evcentric',
  'Dinero gastado': 68.94,
  'cantidad de items': 137,
  'Porcentaje de recomendaciones': '4.38%'},
 'doctr': {'Usuario': 'doctr',
  'Dinero gastado': 99.94999999999999,
  'cantidad de items': 541,
  'Porcentaje de recomendaciones': '1.11%'},
 'maplemage': {'Usuario': 'maplemage',
  'Dinero gastado': 49.97,
  'cantidad de items': 629,
  'Porcentaje de recomendaciones': '0.64%'}}

In [32]:
df_3=data_final[['developer','release_date','title','user_id','recommend','sentiment_analysis']]
df_3=df_3.drop_duplicates(subset=['developer','title','user_id'], keep='first').copy()

In [33]:
df_3['release_date']=df_3['release_date'].astype(int).copy()

In [34]:
def developer_of_year(year: int):
    if year in df_3['release_date'].unique():
        anio=df_3[df_3['release_date']==year].sort_values(by='release_date', ascending=False)
        anio['rating']=anio['recommend']+anio['sentiment_analysis']
        respuesta=anio.groupby(by='developer')['rating'].sum().reset_index()
        respuesta.columns=['Desarrollador','Cantidad']
        respuesta=respuesta.sort_values(by='Cantidad', ascending=False).head(3).reset_index(drop=True)
        respuesta.index+=1
        return respuesta
    else:
        respuesta = f'Error: inserte un año que se encuentre en el rango {df_3['release_date'].min()} - {df_3['release_date'].max()}'
    return respuesta

In [35]:
developer_of_year(2014)

,Desarrollador,Cantidad
1,Edge of Reality,715
2,Coffee Stain Studios,510
3,New World Interactive,426


In [36]:
df_4=data_final[['developer','user_id','sentiment_analysis']]
df_4=df_4.drop_duplicates(subset=['developer','user_id'],keep='first').copy()

In [37]:
def review_developer(dev: str):
    try:
        df=df_4[df_4['developer']==dev]
        positivo=df[df['sentiment_analysis']==2].shape[0]
        negativo=df[df['sentiment_analysis']==0].shape[0]
        respuesta = pd.DataFrame([{'Desarrollador':dev,'Positivo':positivo,'Negativo':negativo}])
        return respuesta
    except:
        return {'No ingreso un valor relevante, o el desarrollador no tiene Resenias'}

In [38]:
review_developer('Valve')

,Desarrollador,Positivo,Negativo
0,Valve,2556,580


In [39]:
df_5=pd.read_parquet(r'C:\Users\Moises\Desktop\Projecto invidual 1\archivos limpios\criterio3.parquet')
df_5.head()

,user_id,playtime_forever,genres,release_date
3568535,76561198040340682,619.0,Action,2013
5669596,76561198044020758,2997.0,Massively Multiplayer,2016
1850439,Dwash,455.0,Casual,2010
5417041,76561198068153515,2903.0,Sports,2014
1589582,76561198067170839,589.0,Strategy,2012


In [40]:
def user_for_genre(gen: str):
    try:
        df=df_5[df_5['genres']==gen]
        usuario= df.groupby('user_id')['playtime_forever'].sum().idxmax(0)
        df_full=df[df['user_id']==usuario]
        horas=df_full.groupby('release_date')['playtime_forever'].sum().reset_index()
        horas.sort_values(by='release_date',ascending=False,inplace=True)
        horas.columns=['Año','Tiempo']
        horas_str = horas.to_string(index=False)
        return f'El usuario con mayor tiempo acumulado jugando juegos del genero {gen} es: {usuario}\n {horas_str}'
    except:
        return 'Error: el valor ingresado no se encuentra en el dataset o valor incorrecto'

In [41]:
mensaje=user_for_genre('Action')
print(mensaje)

El usuario con mayor tiempo acumulado jugando juegos del genero Action es: shinomegami
  Año   Tiempo
2009 291169.0
2003 162201.0
2000 759464.0
